In [17]:
import gensim.downloader as api
from gensim.models import Word2Vec
import numpy as np

In [2]:
#Get common text corpus and train a gensim model on that
dataset = api.load("text8")
model = Word2Vec(dataset)

In [3]:
import glob
import os
import re

In [4]:
#get the name of all the test files
file_list=glob.glob("data/*.txt")
file_list

['data\\Batman.txt',
 'data\\Country.txt',
 'data\\Horse.txt',
 'data\\India.txt',
 'data\\Land.txt',
 'data\\Movie.txt',
 'data\\Pig.txt',
 'data\\White-necked laughingthrush.txt']

In [5]:
name_list=[file.replace("data\\","").replace(".txt","") for file in file_list]
name_list

['Batman',
 'Country',
 'Horse',
 'India',
 'Land',
 'Movie',
 'Pig',
 'White-necked laughingthrush']

In [24]:
content_list=dict()
#remove redundant characters
for name in name_list:
    with open("data/"+name+".txt","r", encoding="utf-8") as f:
        for line in f.readlines():
            line=re.sub('[^A-Za-z.]+', ' ', line)
            content_list[name]=line

In [25]:
#divide the document into sentences
for key, content in content_list.items():
    content_list[key]=content.split(".")

In [26]:
content_list

{'Batman': [' Batman is a fictional superhero appearing in American comic books published by DC Comics',
  ' The character was created by artist Bob Kane and writer Bill Finger and first appeared in Detective Comics in ',
  ' Originally named the Bat Man the character is also referred to by such epithets as the Caped Crusader the Dark Knight and the World s Greatest Detective',
  ' Batman is the secret identity of Bruce Wayne a wealthy American playboy philanthropist and owner of Wayne Enterprises',
  ' Batman originated from an incident in Bruce s childhood after witnessing the murder of his parents Dr',
  ' Thomas Wayne and Martha Wayne he swore vengeance against criminals an oath tempered by a sense of justice',
  ' Bruce trains himself physically and intellectually and crafts a bat inspired persona to fight crime',
  ' Batman operates in the fictional Gotham City with assistance from various supporting characters including his butler Alfred police commissioner Jim Gordon and vigila

In [23]:
content2vec=dict()

#convert the document to sentence vectors

for key, sentences in content_list.items():
    content2vec[key]=[]
    for sentence in sentences:
        if len(sentence)>0:
            centroid=np.zeros((100,))
            sentence=sentence.split()
            for word in sentence:
                try:
                    centroid+=model[word]
                except:
                    pass
            centroid/=len(sentence)
            content2vec[key].append(centroid)

content2vec

C:\Users\vabha\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\Users\vabha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


{'Batman': [array([ 0.02886564,  0.10925334, -0.68183723,  0.00469276,  0.2653463 ,
         -0.40535074, -0.42024544,  0.10751327, -0.37121686, -0.2754195 ,
          0.85069739, -0.14478442, -0.1175284 ,  0.07649485, -0.40369329,
          0.21375242,  0.29372539, -0.94073075,  0.8747114 ,  0.53465065,
          0.15410299,  0.06263314, -0.79434202,  0.37905841, -0.01145629,
         -0.27666414,  0.32561971, -0.88638028,  0.0460059 ,  0.21926715,
         -0.24893177, -0.0497258 , -0.79001976, -0.16310226, -0.08277144,
         -0.45569852, -0.05999756,  1.07722057, -0.827128  , -0.68223342,
         -0.28537402, -0.02627212, -0.93375509,  0.17866512,  0.10285796,
          0.701159  ,  0.01264037,  0.44985075,  0.30934141,  0.23511948,
         -0.31647794,  0.14950363,  0.53069913,  0.26404066, -0.4463818 ,
         -0.17257609,  0.07732028,  0.26866506, -0.1512076 , -0.31931797,
         -0.24205773, -0.11241542, -0.02558003, -0.40279352,  0.95194471,
          0.10528086,  0.238

In [42]:
query="Batman sucks!"
#preprocess the query
query=re.sub('[^A-Za-z]+', ' ', query)
query=query.lower()


#calculate the query vector
query_vec=np.zeros((100,))

for word in query.split():
    try:
        query_vec+=model[word]
    except:
        pass
query_vec/=len(query)

C:\Users\vabha\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [48]:
result_list=dict()

for key, vectors in content2vec.items():
    result_list[key]=0
    for vector in vectors:
        cos_sim = np.dot(vector, query_vec)/(np.linalg.norm(vector)*np.linalg.norm(query_vec))
        result_list[key]+=abs(cos_sim)
            
result_list

C:\Users\vabha\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


{'Batman': 6.524067508543565,
 'Country': 1.5105258110777906,
 'Horse': 0.9727014108078222,
 'India': 3.021837753985444,
 'Land': nan,
 'Movie': 3.944929343045558,
 'Pig': nan,
 'White-necked laughingthrush': 2.023677671674519}